In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import category_encoders as ce
from sklearn import preprocessing 

df_heart = pd.read_csv('heart.csv')
df_heart.head()

In [ ]:
df_heart.shape

In [ ]:
#Проверяем наличие размеченных и не размеченных данных, что бы понять стоит ли дальше проводить анализ
df_heart['target'].value_counts(normalize=True)

#### Как видим выборка состоит из 14 признаков, по 303 пациентам. Как видим 54% пациентом имеющих болезнь сердца. К сожалению не совсем понятно, какая именно болезнь сердца у пациентов, поэтому мы предполагаем что признак болезнь сердца говорит о всех пациентах диагностированных сердечно-сосудистыми заболеваниями. 

In [ ]:
#Переименовываем поля для удобства 
df_heart = df_heart.rename(columns = {
    "age": "возраст",
    "sex": "пол (1 - мужчина, 0 - женщина)",
    "cp": "тип боли в груди (4 значения)",
    "trestbps": "артериальное давление в покое",
    "chol": "холестерин сыворотки в мг/дл",
    "fbs": "уровень сахара в крови натощак > 120 мг/дл",
    "restecg": "результаты электрокардиографии в покое (значения 0,1,2)",
    "thalach": "достигнута максимальная частота сердечных сокращений",
    "exang": "стенокардия, вызванная физической нагрузкой",
    "oldpeak": "депрессия ST, вызванная физической нагрузкой, по сравнению с состоянием покоя",
    "slope": "наклон пикового сегмента ST при нагрузке",
    "ca": "количество крупных сосудов (0-3), окрашенных при флюроскопии",
    "thal":  "дефект, где 3 = нормальный; 6 = фиксированный дефект; 7 = обратимый дефект",
    "old": "Старше 60 лет"
})

In [ ]:
df_heart.head()

In [ ]:
# Проверка null значений
df_heart.isnull().sum()

In [ ]:
# Проверка на дубликаты 
df_heart.duplicated().sum()

In [ ]:
df_heart[df_heart.duplicated(keep=False)]

In [ ]:
df_heart = df_heart.drop_duplicates()

In [ ]:
df_heart.info()

In [ ]:
# Определяем категориальные, количественные, бинарные признаки
for col in df_heart.columns: 
    unique_vals = df_heart[col].nunique()
    if df_heart[col].dtype in ['int64','float64']:
        if unique_vals == 2:
            print(f"{col} - бинарный признак (два значения)")
        elif unique_vals < 5:
            print(f"{col} - категориальный признак (значений менее 5)")
        else:
            print(f"{col} - количественный признак")

In [ ]:
numeric_values = ['возраст','артериальное давление в покое','холестерин сыворотки в мг/дл', 'достигнута максимальная частота сердечных сокращений', 'депрессия ST, вызванная физической нагрузкой, по сравнению с состоянием покоя', 'количество крупных сосудов (0-3), окрашенных при флюроскопии']
categorical_values = ['тип боли в груди (4 значения)','результаты электрокардиографии в покое (значения 0,1,2)','наклон пикового сегмента ST при нагрузке','дефект, где 3 = нормальный; 6 = фиксированный дефект; 7 = обратимый дефект']
binary_values = ['пол (1 - мужчина, 0 - женщина)','уровень сахара в крови натощак > 120 мг/дл','стенокардия, вызванная физической нагрузкой', 'target']

In [ ]:
# Анализ выбросов
fig, axs = plt.subplots(len(numeric_values), 2, figsize=(12, 4* len(numeric_values)))

for i, col in enumerate(numeric_values):

    sns.boxplot(x=df_heart[col], ax = axs[i,0], color = 'skyblue')
    axs[i,0].set_title(f'Boxplot - {col}')

    sns.kdeplot(x =df_heart[col], ax =axs[i,1], color = 'coral')
    axs[i,1].set_title(f'Distribution (KDE) - {col}')

plt.tight_layout()
plt.show()

### Как видно на диаграммах бокс плот и kdeplot, почти все количественные признаки распределены не нормально. Удаление и либо замена признаков в данном кейсе не приемлемо, так как аномальные значения могут указывать на заболевания сердца.

## Детальное изучение выбросов

In [ ]:
# Детальное изучение выбросов
outlier_rows = []
for col in numeric_values:
    Q1 = df_heart[col].quantile(0.25)
    Q3 = df_heart[col].quantile(0.75)
    Median = df_heart[col].median()
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    outliers = df_heart[(df_heart[col]<lower_bound)|(df_heart[col]>upper_bound)]

    for idx, row in outliers.iterrows():
        outliers_info = row.to_dict()
        outliers_info.update({
            "column_with_outlier": col,
            "type": "low" if row[col]<lower_bound else "high",
            "Q1": Q1,
            "Q3": Q3,
            "Median": Median,
            "IQR": IQR,
            "Lower bound": lower_bound,
            "Upper bound": upper_bound
        })
        outlier_rows.append(outliers_info)

outliers_df = pd.DataFrame(outlier_rows)

In [ ]:
outliers_df.head()

In [ ]:
outliers_df['column_with_outlier'].value_counts()

In [ ]:
outliers_df[outliers_df['column_with_outlier']=='количество крупных сосудов (0-3), окрашенных при флюроскопии'][['количество крупных сосудов (0-3), окрашенных при флюроскопии','Median','IQR','Q3','возраст','target']]

### Количество крупных сосудов при флюроскопии может достигать только 3, значение 4 либо аномалия либо не правильно кодировка значения Nan. В таких случаях я бы проконсультировалась с кардиологом. Но думаю что для данного случая, заменю значения на Nan

In [ ]:
df_heart['количество крупных сосудов (0-3), окрашенных при флюроскопии'] = df_heart['количество крупных сосудов (0-3), окрашенных при флюроскопии'].replace(4,np.nan)

In [ ]:
df_heart['количество крупных сосудов (0-3), окрашенных при флюроскопии'].value_counts(normalize=True, dropna=False)

In [ ]:
outliers_df[outliers_df['column_with_outlier']=='артериальное давление в покое'][['артериальное давление в покое',
'Median','IQR','Q3','возраст','target']]

### Показатель артериального давления в покое может быть выше 170 и достигать 200 (как говорит chatgpt), поэтому не похоже что тут есть признак аномалии или ошибочно введенных данных с аномальным выбросом 

In [ ]:
outliers_df[outliers_df['column_with_outlier']=='холестерин сыворотки в мг/дл'][['холестерин сыворотки в мг/дл',
'Median','IQR','Q3','возраст','target']]

### Уровень холестерина выше 300 уже считается опасно высоким, но не невозможным. Для таких случаев я бы проконсультировалась с кардиологом. Но в целом, не похоже не ошибку в данных или аномальным выбросом. Возможно у человека редкая болезнь при которой холестерин опасно высокий

In [ ]:
outliers_df[outliers_df['column_with_outlier']=='депрессия ST, вызванная физической нагрузкой, по сравнению с состоянием покоя'][['депрессия ST, вызванная физической нагрузкой, по сравнению с состоянием покоя','Median','IQR','Q3','возраст','target']]

### Показатель выбросов депрессии ST тоже вызывают подозрение, но не являются невозможными. Поэтому оставляем как есть

In [ ]:
outliers_df[outliers_df['column_with_outlier']
=='достигнута максимальная частота сердечных сокращений'][['достигнута максимальная частота сердечных сокращений','Median','IQR','Q1','Q3','возраст','target']]

### Максимальная частота сердечных сокращений не может быть ниже 90. Больше похоже не ошибку в данных, думаю стоит сделать замену значения на медиану

In [ ]:
median_val = df_heart['достигнута максимальная частота сердечных сокращений'].median()

df_heart.loc[
    df_heart['достигнута максимальная частота сердечных сокращений'] < 90,
    'достигнута максимальная частота сердечных сокращений'
] = median_val

In [ ]:
df_heart = df_heart.sort_values(by='достигнута максимальная частота сердечных сокращений', ascending=True)
df_heart.head()

In [ ]:
#Кодировка категориальных признаков
df_heart[categorical_values] = df_heart[categorical_values].astype('category')

In [ ]:
encoder = ce.OneHotEncoder(cols=categorical_values, drop_invariant=True, use_cat_names=True)
df_encoded=encoder.fit_transform(df_heart[categorical_values])

In [ ]:
df_heart = pd.concat([df_heart.drop(columns=categorical_values), df_encoded], axis=1)

In [ ]:
len(df_heart.columns)

In [ ]:
replacements = {
    'тип боли в груди (4 значения)': 'тип боли',
    'результаты электрокардиографии в покое (значения 0,1,2)': 'экг',
    'наклон пикового сегмента ST при нагрузке': 'наклон ST',
    'дефект, где 3 = нормальный; 6 = фиксированный дефект; 7 = обратимый дефект': 'дефект ST',
    'результаты электрокардиографии в покое (значения 0,1,2)': 'экг'
}

df_heart.rename(
    columns=lambda x: next((x.replace(old, new) for old, new in replacements.items() if old in x), x),
    inplace=True
)


In [ ]:
df_heart.head()

In [ ]:
df_heart.columns

In [ ]:
# Построение матрицы корреляции для корреляционного анализа
plt.figure(figsize=(20,16))
corr_matrix = df_heart.corr(method='spearman').abs() #Так как количественные меры распределены не нормально, стоит избегать корреляции Пирсона
sns.heatmap(corr_matrix, annot = True, cmap='coolwarm', linewidths=.5)
plt.title('Матрица корреляции признаков')

In [ ]:
# Основные влияющие на target переменные
target_corr = df_heart.corr(method='spearman')['target'].abs()
selected_features = target_corr[(target_corr>=0.2)&(target_corr<=0.7)]
selected_features.sort_values(ascending=True)

### Признаки дефект ST_3.0 и дефект ST_2.0 имеют корреляцию выше 80%, а так же наклон ST_2.0 и наклон ST_1.0 имеют корреляцию выше 80% это указывает на признак мультиколлинеарности и может быть проблемой при дальнейшем изучении. Поэтому удаляем один из признаков с меньшим влиянием на target переменную

In [ ]:
selected_features = selected_features.drop(columns=['дефект ST_2.0','наклон ST_2.0'])

In [ ]:
# Финальный список признаков для дальнейшего изучения, по итогам корреляционного анализа
selected_features_df = selected_features.reset_index()
selected_features_df.columns=['Feature','Importance']
selected_features_df.sort_values(by='Importance', ascending=False)

## Дополнительный анализ значимости признаков

In [ ]:
plt.figure(figsize=(16,4))
for feature in numeric_values:
    sns.kdeplot(df_heart[feature], label=feature)
plt.title('Распределение количественных признаков')
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#Разделяем признаки на главный target и влияющие (все кроме target)
X = df_heart.drop(columns=['target'])
y = df_heart['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train,y_train)

In [ ]:
#Извлекаем список важности признаков
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [ ]:
plt.figure(figsize=(16, 8))
ax = sns.barplot(data=feature_importance, x='Importance', y='Feature')

ax.set_title('Топ признаков по методу дерева решений (Random Forest)', fontsize=14)
ax.set_xlabel('Значимость признака')
ax.set_ylabel('Название признака')

# Добавляем подписи к столбцам
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
ax = sns.barplot(data=selected_features_df, x='Importance', y='Feature')

ax.set_title('Топ признаков по итогам корреляционного анализа', fontsize=14)
ax.set_xlabel('Значимость признака')
ax.set_ylabel('Название признака')

# Добавляем подписи к столбцам
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f')

plt.tight_layout()
plt.show()

In [ ]:
selected_features_df.rename(columns={'Importance': 'Correlation'}, inplace=True)
feature_importance_prior_features = feature_importance[feature_importance['Importance']>0.01]

In [ ]:
merged_df = pd.merge(feature_importance_prior_features, selected_features_df, on='Feature', how='outer')

In [ ]:
# итоговый список отобранных признаков для проведения стат тестов
merged_df.sort_values(by='Importance', ascending=False)

### 🧾 Краткий вывод по отобранным признакам

✅ **Сильное совпадение двух подходов:**

Признаки `дефект ST_2.0`, `достигнута максимальная частота сердечных сокращений`, `тип боли_2.0`, `тип боли_1.0`, `наклон ST_2.0` показали высокую **и корреляцию, и важность** — они особенно перспективны для дальнейшего анализа и статистических тестов.

---

🧠 **Признаки с высокой важностью, но без корреляции:**

Например, `артериальное давление в покое`, `холестерин`, `тип боли_3.0` — важны по Random Forest, но не проявляют линейной связи (возможно, у них **нелинейная зависимость**, улавливаемая деревьями решений).

---

📉 **Признаки с высокой корреляцией, но низкой важностью:**

Признак `тип боли_0.0` имеет сильную отрицательную корреляцию, но модель оценивает его вклад как небольшой — возможно, его влияние **перекрывается другими признаками**.

---

🧬 **Клинически значимые признаки:**

Признаки `возраст` и `пол (1 - мужчина, 0 - женщина)` включены в список кандидатов на проведение статтестов несмотря на умеренные значения важности и корреляции. Эти признаки являются **ключевыми в медицинском контексте** и могут выявить значимые различия между группами.

---

⚠️ **Пропуски в корреляции (NaN):**

Это признаки, которые **не участвовали в корреляционном анализе** — вероятно, не показали значимых линейных связей или были исключены ранее.

---

🟢 **Вывод:**

Для проведения **статистических тестов и финального анализа** рекомендовано:
- Ориентироваться на признаки, показавшие значимость по обоим подходам.
- Обязательно включить `возраст` и `пол` как **базовые демографические параметры**.
